In [3]:
import aoc_helpers
day20 = aoc_helpers.AOC(2023,20)
actual = day20.get_inputs().splitlines()

In [10]:
ex_1 = 'broadcaster -> a, b, c\n%a -> b\n%b -> c\n%c -> inv\n&inv -> a'
ex_1 = ex_1.splitlines()

ex_2 ='broadcaster -> a\n%a -> inv, con\n&inv -> b\n%b -> con\n&con -> output'
ex_2 = ex_2.splitlines()

In [57]:
class Module:
    def __init__(self, name, type, dests, sources=None):
        type_dict = {
            '%': 'Flip-Flop',
            '&': 'Conjunction',
        }

        self.name = name

        self.type = type
        self.type_desc = type_dict[type]

        self.dests = dests.split(', ')

        if type == '%':
            self.state = False #Flip-Flops start off
        else: #Must be Conjunction
            state = dict()
            for s in sources:
                state[s] = 'Low'
            self.state = state


    def __repr__(self) -> str:
        return f'{self.type_desc} "{self.name}" -> {self.dests}'
    def receive_pulse(self, pulse):
        source, p_type = pulse

        if self.type == '%' and p_type == 'Low':
            self.state = not self.state
            if self.state: #module now on
                return (self.dests, (self.name, 'High'))
            else:
                return (self.dests, (self.name, 'Low'))
        elif self.type=='&': #Conjunction
            #First update memory
            self.state[source] = p_type
            all_high = all([v=='High' for v in self.state.values()])
            if all_high:
                return (self.dests, (self.name, 'Low'))
            else:
                return (self.dests, (self.name, 'High'))
        return None



In [58]:
def create_modules(inp):
    modules = dict()
    inp = [m.split(' -> ') for m in inp]
    for m in inp:
        if m[0][0] == '%': #Flip-Flop
            modules[m[0][1:]] = Module(
                m[0][1:],
                type='%',
                dests=m[1],
            )
        elif m[0][0] == '&': #Conjunction
            sources = [x[0][1:] for x in inp if m[0][1:] in x[1]]
            modules[m[0][1:]] = Module(
                m[0][1:],
                type='&',
                dests=m[1],
                sources=sources
            )
        else: #Must be broadcaster
            modules[m[0]] = m[1].split(', ')
            

    return modules

In [100]:
def press_btn(modules, p2 = False, log=False):
    high_count = 0
    low_count = 1
    p2_end=False
    Q = []
    Q.append((modules['broadcaster'],('broadcaster', 'Low')))

    while Q:
        
        p = Q.pop(0)
        if log:
            print(f'Module {p[1][0]} sends {p[1][1]} to {p[0]}')

        if p2 and (p[1]==('dd','Low')):
            return True
        for m in p[0]:
            if p[1][1]=='High':
                high_count+=1
            else:
                low_count+=1
            if m in modules:
                out = modules[m].receive_pulse(p[1])
                if out is not None:
                    if log:
                        print(f'    Module {out[1][0]} sends {out[1][1]} to {out[0]}')
                    Q.append(out)
    if not p2:
        return high_count, low_count
    else:
        return False

In [101]:
mods = create_modules(actual)
high_count = 0
low_count = 0

for i in range(1000):
    print(f'Press {i+1} of 1000', end='\r')
    high, low = press_btn(mods, False, False)
    high_count += high
    low_count += low

print('\n')
print(high_count, low_count, high_count*low_count)


Press 1000 of 1000

49189 17754 873301506


In [94]:
day20.submit_a(high_count*low_count)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [99]:
inp = [m.split(' -> ') for m in actual]
sources = [x[0][1:] for x in inp if 'rx' in x[1]]
sources

['dd']

we need to look at when dd sends a 'low' pulse first

In [103]:
mods = create_modules(actual)
btn_press = 0
complete = False

while not complete:
    print(f'Press {i+1}', end='\r')
    btn_press +=1
    complete = press_btn(mods, True)
print('\n')
print(btn_press)